# Ramp of the simulation

In [1]:
#-----Importa paquetes escenciales

from __future__ import division
import hoomd
import hoomd.md

#-----Define las variables importantes

p_prev = 0.8;
p_new = 1.1;
t_max = 5.0 ;
time_steps = 2.0e5;
run_time = time_steps + 1.0e5;

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_5.0_P_0.8_ramp.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.cell()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

#----Integra usando un NPT

pressure = hoomd.variant.linear_interp(points = [(0,p_prev), (time_steps, p_new)])

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max , tau = 1.0, tauP = 5.0, P = pressure);

#-----Guarda la información de la simulación en un archivo .log y en un .gsd

log_file = "T_" + str(t_max) + "_P_" + str(p_new) + "_ramp.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_new) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_new) + "_ramp.gsd"
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite=True); 

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 9000 particles
notice(2): constrain.rigid(): Removing all particles part of rigid bodies (except central particles).Particle tags may change.
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
notice(2): Group "rigid_center" created containing 1000 particles


In [2]:
system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x107770550>),
             ('particles', <hoomd.data.particle_data at 0x115b16160>),
             ('number_density', 0.45645948385216994),
             ('bonds', <hoomd.data.bond_data at 0x115b161d0>),
             ('angles', <hoomd.data.angle_data at 0x115b16240>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x115b162b0>),
             ('impropers', <hoomd.data.dihedral_data at 0x115b16320>),
             ('constraints', <hoomd.data.constraint_data at 0x115b16390>),
             ('pairs', <hoomd.data.bond_data at 0x115b16400>),
             ('timestep', 484000)])

In [3]:
hoomd.run(time_steps)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 484155 / 684000 | TPS 15.3653 | ETA 03:36:46
Time 00:00:20 | Step 484329 / 684000 | TPS 17.1722 | ETA 03:13:47
Time 00:00:30 | Step 484494 / 684000 | TPS 16.3721 | ETA 03:23:05
Time 00:00:40 | Step 484703 / 684000 | TPS 20.8904 | ETA 02:39:00
Time 00:00:50 | Step 484900 / 684000 | TPS 19.5357 | ETA 02:49:51
Time 00:01:00 | Step 485071 / 684000 | TPS 16.9366 | ETA 03:15:45
Time 00:01:10 | Step 485289 / 684000 | TPS 21.7997 | ETA 02:31:55
Time 00:01:20 | Step 485470 / 684000 | TPS 18.0774 | ETA 03:03:02
Time 00:01:30 | Step 485649 / 684000 | TPS 17.7681 | ETA 03:06:03
Time 00:01:40 | Step 485833 / 684000 | TPS 18.3688 | ETA 02:59:48
Time 00:01:50 | Step 486013 / 684000 | TPS 17.8445 | ETA 03:04:55
Time 00:02:00 | Step 486195 / 6

**ERROR**: Particle with unique tag 1457 has NaN for its position.


RuntimeError: Error computing cell list